/usr/local/lib/python3.10/dist-packages/albumentations/core/bbox_utils.py



    bbox=list(bbox)
    for i in range(4):
      if (bbox[i]<0) :
        bbox[i]=0
      elif (bbox[i]>1) :
        bbox[i]=1
    bbox=tuple(bbox)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import albumentations as A
import numpy as np
import matplotlib.pyplot as plt
import torch
import os
import time
import shutil
import torch.nn as nn
import torchvision
import cv2
from google.colab.patches import cv2_imshow
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator
from torch.utils.data import DataLoader, Dataset
from torch.utils.data.sampler import SequentialSampler
from torchvision import utils
from albumentations import (HorizontalFlip, ShiftScaleRotate, VerticalFlip, Normalize, Flip,
                            Compose, GaussNoise)
from torchvision.transforms import ToTensor
from albumentations.pytorch.transforms import ToTensorV2
from torchvision import transforms
import glob as glob

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [ ]:
# directory where all the images are present
# dataset_before
# dataset_after_augmented
# dataset_after_augmented_2
TEST_DIR = "/content/drive/My Drive/thyroid_nodule_detection/Freehand to Bounding Box Conversion Fix/fixed_labels/cropped_dataset/dataset_after_augmented_2/validation/cropped_images/"
TEST_JSON = "/content/drive/My Drive/thyroid_nodule_detection/Freehand to Bounding Box Conversion Fix/fixed_labels/cropped_dataset/dataset_after_augmented_2/validation/cropped_labels.json"



In [ ]:
classes = [
    '__background__', '0', '1' 
]
# classes = [
#     '__background__', '2', '3', '4', '5' 
# ]
num_classes = len(classes)
print(num_classes)

3


In [ ]:
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
# model.to(device)
# get the number of input features 
in_features = model.roi_heads.box_predictor.cls_score.in_features
# replace the pre-trained head with a new one with required number of classes
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
model.to(device).eval()

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=FasterRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


FasterRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=0.0)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=0.0)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=0.0)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256, eps=0.0)
          (relu): ReLU(

In [ ]:
# Load model
checkpoint = torch.load('/content/drive/My Drive/thyroid_nodule_detection/Experiment_5/checkpoints/bestmodel_apr10_with_dataset_after_new_augmented.pt')
model.load_state_dict(checkpoint['state_dict'])
model.to(device).eval()

In [ ]:
test_images = glob.glob(f"{TEST_DIR}/*.jpg")
print(f"Test instances: {len(test_images)}")
# to count the total number of images iterated through
frame_count = 0
# to keep adding the FPS for each image
total_fps = 0
# this will help to create a different color for each class
COLORS = np.random.uniform(0, 255, size=(len(classes), 3))

Test instances: 70


In [ ]:
detection_threshold = 0.5
predicted_bboxes = []
predicted_scores = []
predicted_bboxes_ =[]

for i in range(len(test_images)):
    # get the image file name for saving output later on
    image_name = test_images[i].split(os.path.sep)[-1].split('.')[0]
    image = cv2.imread(test_images[i])
    orig_image = image.copy()
    # BGR to RGB
    image = cv2.cvtColor(orig_image, cv2.COLOR_BGR2RGB).astype(np.float32)
    # make the pixel range between 0 and 1
    image /= 255.0
    # bring color channels to front
    image = np.transpose(image, (2, 0, 1)).astype(np.float32)
    # convert to tensor
    image = torch.tensor(image, dtype=torch.float).cuda()
    # add batch dimension
    image = torch.unsqueeze(image, 0)
    start_time = time.time()
    with torch.no_grad():
        outputs = model(image.to(device))
    end_time = time.time()

    # get the current fps
    fps = 1 / (end_time - start_time)
    # add `fps` to `total_fps`
    total_fps += fps
    # increment frame count
    frame_count += 1
    # load all detection to CPU for further operations
    outputs = [{k: v.to('cpu') for k, v in t.items()} for t in outputs]
    # carry further only if there are detected boxes
    if len(outputs[0]['boxes']) != 0:
        boxes = outputs[0]['boxes'].data.numpy()
        scores = outputs[0]['scores'].data.numpy()
        # filter out boxes according to `detection_threshold`
        boxes = boxes[scores >= detection_threshold].astype(np.int32)
        draw_boxes = boxes.copy()
      
        # get all the predicited class names
        pred_classes = [classes[i] for i in outputs[0]['labels'].cpu().numpy()]
        # draw the bounding boxes and write the class name on top of it
        for j, box in enumerate(draw_boxes):
            class_name = pred_classes[j]
           
          # making prediction bbox for iou
            tensor_bbox = torch.as_tensor([box], dtype=torch.float)
            pred_bbox_item = {
                  "case_id": image_name,
                  "bbox":tensor_bbox,
                  "labels":class_name
            }
            predicted_bboxes.append(pred_bbox_item)

          # image printing  
            color = COLORS[classes.index(class_name)]
            cv2.rectangle(orig_image,
                        (int(box[0]), int(box[1])),
                        (int(box[2]), int(box[3])),
                        color, 2)
            cv2.putText(orig_image, class_name, 
                        (int(box[0]), int(box[1]-5)),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 
                        2, lineType=cv2.LINE_AA)

        cv2_imshow(orig_image)
        cv2.waitKey(1)
        cv2.imwrite(f"/content/drive/My Drive/thyroid_nodule_detection/Experiment_5/inference_outputs/bestmodel_apr1_with_dataset_after/{image_name}.jpg", orig_image)

    print(f"Image {i+1} done...")
    print("Max score {0}".format(scores.max()))
    predicted_scores.append(scores.max())
    print('-'*50)

print('TEST PREDICTIONS COMPLETE')
cv2.destroyAllWindows()
# calculate and print the average FPS
avg_fps = total_fps / frame_count
print(f"Average FPS: {avg_fps:.3f}")
avg_score = sum(predicted_scores) / len(predicted_scores)
print(f"Average scores: {avg_score:.3f}")


print(predicted_bboxes)


# **GET IOU, TEST DATASET, PRECISION AND RECALL**

In [ ]:
class ThyroidNoduleDataset(Dataset):
    def __init__(self, data_frame, image_dir,classes,transforms=None, phase='train'):
        super().__init__()
        self.df = data_frame
        self.image_dir = image_dir
        self.images = data_frame['case_id'].unique()
        self.transforms = transforms
        self.classes = classes

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx] + '.jpg'

        image_arr = cv2.imread(os.path.join(self.image_dir, image), cv2.IMREAD_COLOR)
        image_arr = cv2.cvtColor(image_arr, cv2.COLOR_BGR2RGB).astype(np.float32)
        image_arr /= 255.0

        image_id = str(image.split('.')[0])
        point = self.df[self.df['case_id'] == image_id]
        boxes = point[['x', 'y', 'w', 'h']].values
        boxes[:, 2] = boxes[:, 0] + boxes[:, 2]
        boxes[:, 3] = boxes[:, 1] + boxes[:, 3]

        area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])
        area = torch.as_tensor(area, dtype=torch.float32)

        # there is only one class
        labels = []

        for box in boxes: 
          # labels.append(self.classes.index(str(round(self.df.get('tirads').get(idx)[0]))))
          labels.append(self.classes.index(str(round(self.df.get('labels').get(idx)))))
          # labels.append(self.classes.index(str(round(self.df.get('tirads').get(idx)))))

        
        labels = torch.as_tensor([lbl for lbl in labels])

        # suppose all instances are not crowd
        iscrowd = torch.zeros((point.shape[0],), dtype=torch.int64)

        target = {}
        target['bboxes'] = boxes
        target['labels'] = labels

        if self.transforms:
            sample = {
                'image': image_arr,
                'bboxes': target['bboxes'],
                'labels': target['labels']
            }
            sample = self.transforms(**sample)
            image = sample['image']

        target['bboxes'] = torch.stack(tuple(map(torch.tensor,
                                                zip(*sample['bboxes'])))).permute(1, 0)
        target['boxes'] = target['bboxes'] # later the model needs the 'boxes' naming so keep this copy!

        return image, target, image_id

In [ ]:
def get_test_transform():
    return A.Compose([
        ToTensorV2(p=1.0),
    ], bbox_params={
        'format': 'pascal_voc', 
        'label_fields': ['labels']
    })

In [ ]:
def process_malignancy(number):
  if number <= 3:
    return 0
  else:
    return 1


def process_bbox(df):
    cases = df
    case_list = []
    for case in cases:
        temp_case = {}
        temp_case['case_id'] = case['case_id']
        temp_case['tirads'] = case['labels'][0]
        temp_case['width'] = 400 # should match cropped images
        temp_case['height'] = 350 # should match cropped images
        temp_case["labels"] = process_malignancy(temp_case['tirads'])
        if(len(case['bboxes']) == 1):
          temp_case['x'] = case['bboxes'][0][0]
          temp_case['y'] = case['bboxes'][0][1]
          temp_case['w'] = case['bboxes'][0][2]
          if len(case['bboxes'][0]) < 4:
            temp_case['h'] = 1 # bounding box should have height which is greater than 0 if invalid to no break training
          else:
            temp_case['h'] = case['bboxes'][0][3]
          case_list.append(temp_case)
        else:
          i = len(case['bboxes'])-1
          while i >= 0:
            temp_case['tirads'] = case['labels'][i]
            temp_case["labels"] = process_malignancy(temp_case['tirads'])
            temp_case['x'] = case['bboxes'][i][0]
            temp_case['y'] = case['bboxes'][i][1]
            temp_case['w'] = case['bboxes'][i][2]
            if len(case['bboxes'][0]) < 4:
              temp_case['h'] = 1 # bounding box should have height which is greater than 0 if invalid to no break training
            else:
              temp_case['h'] = case['bboxes'][i][3]
            case_list.append(temp_case)
            i -= 1         
    return pd.DataFrame(case_list)

In [ ]:
temp_test_df = pd.read_json(TEST_JSON)
test_df = process_bbox(temp_test_df['cases'])
test_data = ThyroidNoduleDataset(test_df, TEST_DIR,classes,get_test_transform(),  phase='validation')

In [ ]:
#predefine ground truth bounding boxes
ground_truth_bboxes = []
for data in test_data:
  tensor_bbox = torch.tensor(data[1]['bboxes'], dtype=torch.float)
  ground_truth_bboxes.append({
      'case_id': data[2],
      'bbox':tensor_bbox,
      'label': data[1]['labels'][0]
  })

In [ ]:
print('PREDICTED BBOXES:', predicted_bboxes)
print(len(predicted_bboxes))
print('GROUND TRUTH BBOXES:',ground_truth_bboxes)
print(len(ground_truth_bboxes))

In [ ]:
# Get iou
from torchvision import ops

ious = []
for i,item_i in enumerate(predicted_bboxes):
  for j,item_j in enumerate(ground_truth_bboxes):
    if item_i['case_id'] == item_j['case_id']:
      iou = ops.box_iou(item_j['bbox'],item_i['bbox'])
      ious.append(iou)
      print(f'{i+1}.TEST CASE IOU : {iou.numpy()[0][0]}', 
            {item_i['case_id']},
            {item_j['case_id']})

# all_iou = np.array(thresholds, dtype=np.float32)
# print(f'AVARAGE IOU : {np.average(all_iou)}')
avg_iou = sum(ious) / len(ground_truth_bboxes)
avg_iou = avg_iou.numpy()[0][0]
print(f'AVARAGE IOU : {avg_iou * 100}')

In [ ]:
import numpy
import sklearn.metrics
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import PrecisionRecallDisplay
from sklearn.metrics import precision_recall_curve

In [ ]:
threshold = detection_threshold

In [ ]:
# print(y_true)
# print(y_pred)


In [ ]:
y_true =  [1 if test_df['labels'][ind] == 1 else 0 for ind in test_df.index]
y_pred = [1 if score >= threshold else 0 for score in predicted_scores]



# Confusion Matrix (From Left to Right & Top to Bottom: True Positive, False Negative, False Positive, True Negative)
confusion_matrix = numpy.flip(sklearn.metrics.confusion_matrix(y_true, y_pred,labels=[0,1],normalize='all'))
cm_display = ConfusionMatrixDisplay(confusion_matrix).plot()

In [ ]:
y_scores = predicted_scores

In [ ]:
PrecisionRecallDisplay.from_predictions(y_true, y_scores)

plt.show()

In [ ]:
from sklearn.metrics import classification_report
target_names = ['benign 0', 'malignant 1']
print(classification_report(y_true, y_pred, target_names=target_names))

              precision    recall  f1-score   support

    benign 0       0.26      0.30      0.28        20
 malignant 1       0.73      0.69      0.70        54

    accuracy                           0.58        74
   macro avg       0.49      0.49      0.49        74
weighted avg       0.60      0.58      0.59        74



In [ ]:
from sklearn.metrics import roc_curve
from sklearn.metrics import RocCurveDisplay

fpr, tpr, _ = roc_curve(y_true, y_scores, pos_label="malignant")
roc_display = RocCurveDisplay(fpr=fpr, tpr=tpr).plot()

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, average_precision_score
print('Precision: %.3f' % precision_score(y_true, y_pred, pos_label=1))
print('Recall: %.3f' % recall_score(y_true, y_pred,pos_label=1))
print('Accuracy: %.3f' % accuracy_score(y_true, y_pred))
print('F1 Score: %.3f' % f1_score(y_true, y_pred,pos_label=1))
print('Average Precesion Score: %.3f' % average_precision_score(y_true, y_pred))
print('Average Precesion Score: %.3f' % average_precision_score(y_true, y_pred,pos_label=0))

AP_M=average_precision_score(y_true, y_pred,pos_label=1)
AP_B=average_precision_score(y_true, y_pred,pos_label=0)

ap_scores = [AP_M,AP_B]

# Calculate mAP score
mAP_score = np.mean(ap_scores)

print("mAP score:%.3f" %  mAP_score)

